# Tranformer

In [2]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

## 1. Механизм внимания (Scaled Dot-Product Attention)

In [3]:
class ScaledDotProductAttention(nn.Module):
    """Attention"""

    def __init__(self, d_k):
        """Args:
        d_k (_type_): Размерность ключа
        """
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, mask=None):
        # Скалярное произведение Q и K
        scores = torch.matmul(Q, K.transponse(-2, -1)) / math.sqrt(self.d_k)

        # Применяем маску, если она есть
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attention = F.softmax(scores, dim=-1)

        # Скалярное произведение attention и V
        output = torch.matmul(attention, V)
        return output, attention

## Multihead attention

In [4]:
import torch.nn as nn


class MultiHeadAttention(nn.Module):
    """Multi-head attention"""

    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heada = num_heads
        self.d_k = d_model // num_heads

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)

        self.fc_out = nn.Linear(d_model, d_model)

    def forward(Q, K, V, mask=None):
        batch_size = Q.shape[0]

        # Линейные преобразования для Q, K, V.
        Q = self.query(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.query(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.query(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Применим механизм внимания
        attn_output, _ = ScaledDotProductAttention(self.d_k)(Q, K, V, mask)

        # Объединяем головы
        attn_output = (
            attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        )
        output = self.fc_out(attn_output)
        return output